In [10]:
import pandas as pd
df_train=pd.read_csv('../files/input/train_data.csv.zip',index_col=False,compression='zip')
df_test=pd.read_csv('../files/input/test_data.csv.zip',index_col=False,compression='zip')

#### Paso 1.
##### Realice la limpieza de los datasets:
- Renombre la columna "default payment next month" a "default".
- Remueva la columna "ID".
- Elimine los registros con informacion no disponible.
- Para la columna EDUCATION, valores > 4 indican niveles superiores  de educación, agrupe estos valores en la categoría "others".
- Renombre la columna "default payment next month" a "default"
- Remueva la columna "ID".

In [11]:
df_train.rename(columns={"default payment next month":"default"},inplace=True)
df_test.rename(columns={"default payment next month":"default"},inplace=True)

df_train.drop(columns={"ID"},inplace=True)
df_test.drop(columns={"ID"},inplace=True)

# df_train.columns

df_train.dropna(inplace=True)
df_test.dropna(inplace=True)
print(df_train.isna().sum())
print(df_test.isna().sum())
df_train["EDUCATION"].apply(lambda x: 4 if x > 4 else x)
df_test["EDUCATION"].apply(lambda x: 4 if x > 4 else x)
# df_train["MARRIAGE"].apply(lambda x: 4 if x > 4 else x)
# df_test["MARRIAGE"].apply(lambda x: 4 if x > 4 else x)

print(df_train["EDUCATION"].value_counts())
df_train = df_train.loc[df_train["EDUCATION"] != 0]
df_test = df_test.loc[df_test["EDUCATION"]!=0]
df_train = df_train.loc[df_train["MARRIAGE"]!=0]
df_test = df_test.loc[df_test["MARRIAGE"]!=0]


LIMIT_BAL    0
SEX          0
EDUCATION    0
MARRIAGE     0
AGE          0
PAY_0        0
PAY_2        0
PAY_3        0
PAY_4        0
PAY_5        0
PAY_6        0
BILL_AMT1    0
BILL_AMT2    0
BILL_AMT3    0
BILL_AMT4    0
BILL_AMT5    0
BILL_AMT6    0
PAY_AMT1     0
PAY_AMT2     0
PAY_AMT3     0
PAY_AMT4     0
PAY_AMT5     0
PAY_AMT6     0
default      0
dtype: int64
LIMIT_BAL    0
SEX          0
EDUCATION    0
MARRIAGE     0
AGE          0
PAY_0        0
PAY_2        0
PAY_3        0
PAY_4        0
PAY_5        0
PAY_6        0
BILL_AMT1    0
BILL_AMT2    0
BILL_AMT3    0
BILL_AMT4    0
BILL_AMT5    0
BILL_AMT6    0
PAY_AMT1     0
PAY_AMT2     0
PAY_AMT3     0
PAY_AMT4     0
PAY_AMT5     0
PAY_AMT6     0
default      0
dtype: int64
EDUCATION
2    9759
1    7480
3    3427
5     187
4      98
6      40
0       9
Name: count, dtype: int64


#### Paso 2.
##### Divida los datasets en x_train, y_train, x_test, y_test.

In [12]:
X_train = df_train.drop(columns={"default"})
X_test = df_test.drop(columns={"default"})
y_train =df_train["default"]
y_test =df_test["default"]

#### Paso 3.
##### Cree un pipeline para el modelo de clasificación. Este pipeline debe contener las siguientes capas:
- Transforma las variables categoricas usando el método one-hot-encoding.
- Descompone la matriz de entrada usando componentes principales.El pca usa todas las componentes.
- Escala la matriz de entrada al intervalo [0, 1].
- Selecciona las K columnas mas relevantes de la matrix de entrada.
- Ajusta una red neuronal tipo MLP.

In [19]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
numerical_features= ['LIMIT_BAL', 'AGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6' ]
transformer = ColumnTransformer(transformers=[('ohe',OneHotEncoder(),['SEX','MARRIAGE','EDUCATION']),
                                              ('scaler', StandardScaler(),numerical_features)], 
                                remainder='passthrough')

selectkbest = SelectKBest(score_func=f_classif)
pca = PCA()
# red neuronal tipo mlp
mlp = MLPClassifier(max_iter = 1000, random_state=420)
pipeline = Pipeline(steps=[('transformer', transformer),
                           ('selector', selectkbest),
                           ('pca',pca),
                           ('model', mlp)])


In [20]:
from sklearn.model_selection import  GridSearchCV
from sklearn.metrics import balanced_accuracy_score


# param_grid = {
#     'selector__k': [10, 15],                # Número de características seleccionadas
#     'model__hidden_layer_sizes': [(50,), (50, 50)],  # Arquitectura de la red
#     'model__activation': ['relu'],         # Función de activación principal
#     'model__solver': ['adam', 'sgd'],      # Optimizadores comunes
#     'model__alpha': [0.001, 0.01],         # Regularización
#     'model__max_iter': [300, 500],         # Iteraciones máximas
# }

param_grid = {
    'pca__n_components': [20],
    'selector__k': [20],
    'model__hidden_layer_sizes': [(50,30,40,60)],
    'model__alpha': [0.256],
    'model__learning_rate': ['adaptive'],
    'model__activation': ['relu'],
    'model__solver': ['adam'],
    'model__learning_rate_init': [0.001]
}


# param_grid = {
#     'selector__k': [15],    #15    15396       # Hiperparámetro del selector (por ejemplo, SelectKBest)
#     'model__hidden_layer_sizes': [(100, 100, 1000)], #(50, 50) # Hiperparámetros del MLP
#     'model__activation': ['logistic'], #relu  
#     'model__solver': ['adam','sgd'], #'adam','sgd'
#     'model__alpha': [0.00001], #0.00001  15399
#     'model__max_iter': [200,500, 1000], #300
#     'model__early_stopping': [True],
#     # 'model__learning_rate': ['constant', 'adaptive', 'invscaling'],


# }
# Crear el objeto GridSearchCV con validación cruzada de 10 divisiones y precisión balanceada 
grid_search = GridSearchCV(
    estimator = pipeline,
    param_grid = param_grid,
    scoring = "balanced_accuracy", 
    cv=10,
    n_jobs=-1,
    )

# Entrenar el modelo con GridSearchCV 
grid_search.fit(X_train, y_train)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('transformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe',
                                                                         OneHotEncoder(),
                                                                         ['SEX',
                                                                          'MARRIAGE',
                                                                          'EDUCATION']),
                                                                        ('scaler',
                                                                         StandardScaler(),
                                                                         ['LIMIT_BAL',
                                                                          'AGE',
                                                                          'PAY_0',
                                                                          'PAY_2',
                                                                          'PAY_3',
                                                                          'PAY_4',
                                                                          'PAY_5',
                                                                          'PAY_6',
                                                                          'BILL_AMT1',
                                                                          'BILL_AMT2',
                                                                          'BILL_AMT3',
                                                                          'BILL_AMT4',
                                                                          'BILL_AMT5',
                                                                          'BILL_AMT6',
                                                                          'PAY_AMT1',
                                                                          'P...
                                       ('pca', PCA()),
                                       ('model',
                                        MLPClassifier(max_iter=1000,
                                                      random_state=420))]),
             n_jobs=-1,
             param_grid={'model__activation': ['relu'], 'model__alpha': [0.256],
                         'model__hidden_layer_sizes': [(50, 30, 40, 60)],
                         'model__learning_rate': ['adaptive'],
                         'model__learning_rate_init': [0.001],
                         'model__solver': ['adam'], 'pca__n_components': [20],
                         'selector__k': [20]},
             scoring='balanced_accuracy')

In [21]:
# Obtener los mejores parámetros y el mejor modelo 
best_params = grid_search.best_params_ #coge los mejores parametros del estimador
best_score = grid_search.best_score_
print(best_params)
model= grid_search

{'model__activation': 'relu', 'model__alpha': 0.256, 'model__hidden_layer_sizes': (50, 30, 40, 60), 'model__learning_rate': 'adaptive', 'model__learning_rate_init': 0.001, 'model__solver': 'adam', 'pca__n_components': 20, 'selector__k': 20}


In [22]:
##paso 5
import gzip
import pickle
import os
# Crear carpeta si no existe
output_dir = '../files/models'
os.makedirs(output_dir, exist_ok=True)

compressed_model_path = '../files/models/model.pkl.gz'

with gzip.open(compressed_model_path, 'wb') as f:
    pickle.dump(model, f)

In [23]:
## paso 6
from sklearn.metrics import precision_score, recall_score, f1_score
import json

y_train_pred = model.predict(X_train) 
y_test_pred = model.predict(X_test)

# Calcular las métricas para el conjunto de entrenamiento, en la primera va el nombre del dataset y en la segunda el train 
train_metrics = { 'type': "metrics",
    'dataset': 'train', 
                'precision': float(precision_score(y_train, y_train_pred, average='weighted')),
                'balanced_accuracy': float(balanced_accuracy_score(y_train, y_train_pred)), 
                'recall': float(recall_score(y_train, y_train_pred, average='weighted')), 
                'f1_score': float(f1_score(y_train, y_train_pred, average='weighted')) }

# Calcular las métricas para el conjunto de prueba 
test_metrics = { 'type': "metrics",
    'dataset': 'test', 
                'precision': float(precision_score(y_test, y_test_pred, average='weighted')), 
                'balanced_accuracy': float(balanced_accuracy_score(y_test, y_test_pred)),
                'recall': float(recall_score(y_test, y_test_pred, average='weighted')), 
                'f1_score': float(f1_score(y_test, y_test_pred, average='weighted')) }

print("Metricas de entrenamiento")
print(json.dumps(train_metrics, indent=4))
print("Metricas de prueba")
print(json.dumps(test_metrics, indent=4))

Metricas de entrenamiento
{
    "type": "metrics",
    "dataset": "train",
    "precision": 0.8071567513995938,
    "balanced_accuracy": 0.6692644694061998,
    "recall": 0.8223166133727867,
    "f1_score": 0.803150610538164
}
Metricas de prueba
{
    "type": "metrics",
    "dataset": "test",
    "precision": 0.8147447332498727,
    "balanced_accuracy": 0.6719029580440464,
    "recall": 0.8306047444036084,
    "f1_score": 0.8132121340716103
}


Metricas de entrenamiento
{
    "type": "metrics",
    "dataset": "train",
    "precision": 0.8090676936415361,
    "balanced_accuracy": 0.658232808145305,
    "recall": 0.8228893237245263,
    "f1_score": 0.799515046476117
}
Metricas de prueba
{
    "type": "metrics",
    "dataset": "test",
    "precision": 0.8151033022356751,
    "balanced_accuracy": 0.6624624345511485,
    "recall": 0.8308274863570554,
    "f1_score": 0.8102971329938454
}
Métricas guardadas en ../files/output/metrics.json

In [24]:
############  paso 7
from sklearn.metrics import confusion_matrix
# Matriz de confusión para el conjunto de entrenamiento
cm_train = confusion_matrix(y_train, y_train_pred)

# Matriz de confusión para el conjunto de prueba
cm_test = confusion_matrix(y_test, y_test_pred)
# Crear la matriz de confusión formateada como diccionario
metrics = []

# Matriz de confusión para entrenamiento
metrics = [
    {
        'type': 'cm_matrix',
        'dataset': 'train',
        'true_0': {"predicted_0": int(cm_train[0][0]), "predicted_1": int(cm_train[0][1])},
        'true_1': {"predicted_0": int(cm_train[1][0]), "predicted_1": int(cm_train[1][1])}
    },
    {
        'type': 'cm_matrix',
        'dataset': 'test',
        'true_0': {"predicted_0": int(cm_test[0][0]), "predicted_1": int(cm_test[0][1])},
        'true_1': {"predicted_0": int(cm_test[1][0]), "predicted_1": int(cm_test[1][1])}
    }
    
]

# Crear carpeta si no existe
output_dir = '../files/output'
os.makedirs(output_dir, exist_ok=True)

# Guardar las métricas en un archivo JSON
output_path = '../files/output/metrics.json'
with open(output_path, 'w') as f:  # Usar 'w' para comenzar con un archivo limpio
        f.write(json.dumps(train_metrics) + '\n')
        f.write(json.dumps(test_metrics) + '\n')
        for metric in metrics:
            f.write(json.dumps(metric) + '\n')


print(f"Métricas guardadas en {output_path}")

Métricas guardadas en ../files/output/metrics.json
